POSTagging dengan metode klasifikasi konvensional. 

Metode klasifikasi yang digunakan pada contoh ini adalah *Decision Tree*.


sumber: https://nlpforhackers.io/training-pos-tagger/

Import library yang dibutuhkan

In [48]:
import nltk
nltk.download('punkt')

from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Baca data latih.
Data latih diambil dari cuplikan dataset yang digunakan di https://github.com/kmkurn/id-pos-tagging

In [49]:
text = []
sentence_1 = 'Situasi/NN yang/SC kini/NN kita/PRP alami/JJ ,/Z memang/RB menggelisahkan/VB ./Z'
sentence_2 = 'Target/NN ekonomi/NN yang/SC dicanangkan/VB pemerintah/NN bisa/MD tercapai/VB'
sentence_3 = 'Pertemuan/NN dua/CD hari/NN yang/SC berlangsung/VB di/IN Doha/NNP ,/Z disponsori/VB oleh/IN Komite/NNP '
sentence_3 += ' Pelatihan/NNP Hak-hak/NNP Yang/NNP Tak/NNP Bisa/NNP Dicabut/NNP dari/NNP Rakyat/NNP Palestina/NNP ,/Z'
sentence_4 = 'Sedangkan/CC penerimaan/NN pemerintah/NN ,/Z kata/VB -nya/PRP ,/Z dianggap/VB masih/MD positif/JJ'

text.append(sentence_1)
text.append(sentence_2)
text.append(sentence_3)
text.append(sentence_4)

In [50]:
print(text)

['Situasi/NN yang/SC kini/NN kita/PRP alami/JJ ,/Z memang/RB menggelisahkan/VB ./Z', 'Target/NN ekonomi/NN yang/SC dicanangkan/VB pemerintah/NN bisa/MD tercapai/VB', 'Pertemuan/NN dua/CD hari/NN yang/SC berlangsung/VB di/IN Doha/NNP ,/Z disponsori/VB oleh/IN Komite/NNP  Pelatihan/NNP Hak-hak/NNP Yang/NNP Tak/NNP Bisa/NNP Dicabut/NNP dari/NNP Rakyat/NNP Palestina/NNP ,/Z', 'Sedangkan/CC penerimaan/NN pemerintah/NN ,/Z kata/VB -nya/PRP ,/Z dianggap/VB masih/MD positif/JJ']


Baca dataset menjadi 2 bagian list: kata-kata dalam kalimat dan tag

In [51]:
sentences = [] # list untuk menampung kalimat dan kata-kata di dalamnya
tags = [] # list untuk menampung tag

for i in range(0,len(text)):
  tokens = text[i].split()
  sent = []
  tag = []
  for j in range(0,len(tokens)):
    pair = tokens[j].split('/')
    sent.append(pair[0])
    tag.append(pair[1])
  sentences.append(sent)
  tags.append(tag)

In [52]:
print(sentences)

[['Situasi', 'yang', 'kini', 'kita', 'alami', ',', 'memang', 'menggelisahkan', '.'], ['Target', 'ekonomi', 'yang', 'dicanangkan', 'pemerintah', 'bisa', 'tercapai'], ['Pertemuan', 'dua', 'hari', 'yang', 'berlangsung', 'di', 'Doha', ',', 'disponsori', 'oleh', 'Komite', 'Pelatihan', 'Hak-hak', 'Yang', 'Tak', 'Bisa', 'Dicabut', 'dari', 'Rakyat', 'Palestina', ','], ['Sedangkan', 'penerimaan', 'pemerintah', ',', 'kata', '-nya', ',', 'dianggap', 'masih', 'positif']]


In [53]:
print(tags)

[['NN', 'SC', 'NN', 'PRP', 'JJ', 'Z', 'RB', 'VB', 'Z'], ['NN', 'NN', 'SC', 'VB', 'NN', 'MD', 'VB'], ['NN', 'CD', 'NN', 'SC', 'VB', 'IN', 'NNP', 'Z', 'VB', 'IN', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'Z'], ['CC', 'NN', 'NN', 'Z', 'VB', 'PRP', 'Z', 'VB', 'MD', 'JJ']]


Fungsi untuk ekstraksi fitur.

Perhatikan bahwa di contoh ini kata tidak diubah menjadi huruf kecil semua (*lower case*), namun juga tidak ada fitur yang terkait dengan penggunaan huruf kapital.

In [54]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    #print("sentence index = ")
    #print(sentence[index])
    prefix_1 = ''
    prefix_2 = ''
    suffix_1 = ''
    suffix_2 = ''
    if (len(sentence[index])>2):
      prefix_1 = sentence[index][0]
      prefix_2 = sentence[index][:2]
      suffix_1 = sentence[index][-1]
      suffix_2 = sentence[index][-2:]
    return {
        'word': sentence[index],
        'prefix-1': prefix_1,
        'prefix-2': prefix_2,        
        'suffix-1': suffix_1,
        'suffix-2': suffix_2,        
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
    }

Tes ekstraksi fitur dari sebuah kalimat

In [55]:
sent_example = 'Situasi yang kini kita alami'
sent_example_tokens = sent_example.split()
for i in range (0,len(sent_example_tokens)):
  print('fitur kata ',sent_example_tokens[i],'adalah sebagai berikut:')
  token_feature = features(sent_example_tokens,i)
  print(token_feature)

fitur kata  Situasi adalah sebagai berikut:
{'word': 'Situasi', 'prefix-1': 'S', 'prefix-2': 'Si', 'suffix-1': 'i', 'suffix-2': 'si', 'prev_word': '', 'next_word': 'yang'}
fitur kata  yang adalah sebagai berikut:
{'word': 'yang', 'prefix-1': 'y', 'prefix-2': 'ya', 'suffix-1': 'g', 'suffix-2': 'ng', 'prev_word': 'Situasi', 'next_word': 'kini'}
fitur kata  kini adalah sebagai berikut:
{'word': 'kini', 'prefix-1': 'k', 'prefix-2': 'ki', 'suffix-1': 'i', 'suffix-2': 'ni', 'prev_word': 'yang', 'next_word': 'kita'}
fitur kata  kita adalah sebagai berikut:
{'word': 'kita', 'prefix-1': 'k', 'prefix-2': 'ki', 'suffix-1': 'a', 'suffix-2': 'ta', 'prev_word': 'kini', 'next_word': 'alami'}
fitur kata  alami adalah sebagai berikut:
{'word': 'alami', 'prefix-1': 'a', 'prefix-2': 'al', 'suffix-1': 'i', 'suffix-2': 'mi', 'prev_word': 'kita', 'next_word': ''}


Fungsi untuk transformasi format dataset

In [56]:
def transform_to_dataset(sentences, tags):
    X, y = [], []
 
    for sentence_idx in range(len(sentences)):
        for index in range(len(sentences[sentence_idx])):
            X.append(features(sentences[sentence_idx], index))
            y.append(tags[sentence_idx][index])
 
    return X, y

Split dataset menjadi data training dan testing

Contoh di sini digunakan rasio 75:25 untuk perbandingan data training:testing

In [57]:
cutoff = int(.75 * len(sentences))
training_sentences = sentences[:cutoff]
test_sentences = sentences[cutoff:]
training_tags = tags[:cutoff]
test_tags = tags[cutoff:]

In [58]:
print(test_sentences)

[['Sedangkan', 'penerimaan', 'pemerintah', ',', 'kata', '-nya', ',', 'dianggap', 'masih', 'positif']]


In [59]:
print(test_tags)

[['CC', 'NN', 'NN', 'Z', 'VB', 'PRP', 'Z', 'VB', 'MD', 'JJ']]


Training

In [60]:
X, y = transform_to_dataset(training_sentences, training_tags)


from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
 
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', tree.DecisionTreeClassifier(criterion='entropy'))
])
clf.fit(X, y)   
 
print('Training completed')

Training completed


Testing

In [61]:
X_test, y_test = transform_to_dataset(test_sentences, test_tags)
 
print("Accuracy:")
print(clf.score(X_test, y_test))

Accuracy:
0.3


Tes pada kalimat uji baru

In [62]:
from nltk import word_tokenize

def pos_tag(sentence):
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    #return zip(sentence, tags)
    return tags
 
print(pos_tag(word_tokenize('situasi hari ini berlangsung positif')))

['NN' 'NNP' 'NNP' 'NNP' 'NNP']
